In [1]:
import os

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, precision_score, recall_score

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

import copy

import time

import torch.optim as optim
#from dataset import train_dataloader, val_dataloader
#from dataset import train_dataset, val_dataset
from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import numpy as np

from sklearn.model_selection import cross_val_score

import pickle
from sklearn.preprocessing import MinMaxScaler

pd.options.display.max_columns = None

from sklearn.linear_model import LogisticRegression

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#main_dir = os.path.abspath('..') # os.path.join(os.getcwd())
#data_dir = os.path.join(main_dir, "Data")
#shapefiles_dir = os.path.join(main_dir, data_dir, "shapefiles")

In [3]:
# Main files:
# precovid_crime_micro_geo_11_29_2021
# covid_crime_micro_geo_11_29_2021

pre_covid_data = pd.read_csv('https://raw.githubusercontent.com/joannechoi/DC-Crime-Analysis/main/Data/Merged/precovid_crime_micro_geo_11_29_2021.csv', index_col=False)
covid_data = pd.read_csv('https://raw.githubusercontent.com/joannechoi/DC-Crime-Analysis/main/Data/Merged/covid_crime_micro_geo_11_29_2021.csv', index_col=False)

del pre_covid_data["SHIFT"]
del pre_covid_data["OFFENSE"]
del pre_covid_data['LATITUDE']
del pre_covid_data['LONGITUDE']
del pre_covid_data["ward_1"]
del pre_covid_data["ward_2"]
del pre_covid_data["ward_3"]
del pre_covid_data["ward_4"]
del pre_covid_data["ward_5"]
del pre_covid_data["ward_6"]
del pre_covid_data["ward_7"]
del pre_covid_data["ward_8"]
del pre_covid_data[covid_data.columns[0]]
# del pre_covid_data['date']

del covid_data["SHIFT"]
del covid_data["OFFENSE"]
del covid_data['LATITUDE']
del covid_data['LONGITUDE']
del covid_data["ward_1"]
del covid_data["ward_2"]
del covid_data["ward_3"]
del covid_data["ward_4"]
del covid_data["ward_5"]
del covid_data["ward_6"]
del covid_data["ward_7"]
del covid_data["ward_8"]
del covid_data[covid_data.columns[0]]
# del covid_data['date']

pre_covid_data['WARD'] = pre_covid_data['WARD'].astype(int)
covid_data['WARD'] = covid_data['WARD'].astype(int)

In [4]:
pre_covid_data

,WARD,date,total_cases,daily_cases,daily_death,OUT_POSITIVE_QUARANTINE_PSMPD,MO,TU,WE,TH,FR,SA,SU,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ARSON,ASSAULT W/DANGEROUS WEAPON,BURGLARY,HOMICIDE,MOTOR VEHICLE THEFT,ROBBERY,SEX ABUSE,THEFT F/AUTO,THEFT/OTHER,DAY,EVENING,MIDNIGHT,distance_metro,distance_nightclubs,distance_worship
0,5,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1.127080,0.674769,0.077614
1,6,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.191281,0.079607,0.049559
2,6,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0.083639,0.124886,0.068719
3,6,2017-10-27,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.135380,0.296590,0.074763
4,5,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0.084503,0.849526,0.047963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104354,6,2020-03-01,0.0,0.0,0.0,0.0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0.138408,0.513213,0.030329
104355,5,2009-12-01,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1.039210,0.809993,0.124818
104356,8,2019-02-18,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0.559830,0.569863,0.005769
104357,5,2004-02-02,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0.252021,0.904808,0.062211


In [5]:
covid_data

,WARD,date,total_cases,daily_cases,daily_death,OUT_POSITIVE_QUARANTINE_PSMPD,MO,TU,WE,TH,FR,SA,SU,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ARSON,ASSAULT W/DANGEROUS WEAPON,BURGLARY,HOMICIDE,MOTOR VEHICLE THEFT,ROBBERY,SEX ABUSE,THEFT F/AUTO,THEFT/OTHER,DAY,EVENING,MIDNIGHT,distance_metro,distance_nightclubs,distance_worship
0,6,2020-09-01,14077.0,28.0,1.0,80.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0.189878,0.352657,0.081604
1,8,2020-09-01,14077.0,28.0,1.0,80.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0.585539,1.073603,0.156339
2,4,2020-09-01,14077.0,28.0,1.0,80.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0.490429,0.876416,0.009941
3,6,2020-08-31,14049.0,57.0,0.0,87.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0.229824,0.360878,0.051530
4,2,2020-09-01,14077.0,28.0,1.0,80.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0.063730,0.035825,0.076037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43159,5,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0.418831,0.078911,0.159204
43160,7,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.718883,0.790578,0.034208
43161,5,2021-02-03,37365.0,166.0,6.0,106.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.345018,0.900916,0.097899
43162,8,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0.442868,0.972997,0.004663


In [6]:
# Baseline:
x_PRE_COVID_baseline_not_scaled = pre_covid_data.loc[:, pre_covid_data.columns != 'WARD']
del x_PRE_COVID_baseline_not_scaled["total_cases"]
del x_PRE_COVID_baseline_not_scaled["daily_cases"]
del x_PRE_COVID_baseline_not_scaled["daily_death"]
del x_PRE_COVID_baseline_not_scaled["OUT_POSITIVE_QUARANTINE_PSMPD"]

x_COVID_baseline_not_scaled = covid_data.loc[:, pre_covid_data.columns != 'WARD']

# Experiment:
x_PRE_COVID_experiment_not_scaled = covid_data.loc[:, pre_covid_data.columns != 'WARD']
del x_PRE_COVID_experiment_not_scaled["total_cases"]
del x_PRE_COVID_experiment_not_scaled["daily_cases"]
del x_PRE_COVID_experiment_not_scaled["daily_death"]
del x_PRE_COVID_experiment_not_scaled["OUT_POSITIVE_QUARANTINE_PSMPD"]

x_COVID_experiment_not_scaled = pre_covid_data.loc[:, pre_covid_data.columns != 'WARD']

# Labels:
y_PRE_COVID_base = pre_covid_data['WARD']
y_COVID_base = covid_data['WARD']

y_PRE_COVID_experiment = covid_data['WARD']
y_COVID_experiment = pre_covid_data['WARD']

In [7]:
x_PRE_COVID_baseline_not_scaled

,date,MO,TU,WE,TH,FR,SA,SU,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ARSON,ASSAULT W/DANGEROUS WEAPON,BURGLARY,HOMICIDE,MOTOR VEHICLE THEFT,ROBBERY,SEX ABUSE,THEFT F/AUTO,THEFT/OTHER,DAY,EVENING,MIDNIGHT,distance_metro,distance_nightclubs,distance_worship
0,2017-10-28,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1.127080,0.674769,0.077614
1,2017-10-28,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.191281,0.079607,0.049559
2,2017-10-28,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0.083639,0.124886,0.068719
3,2017-10-27,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.135380,0.296590,0.074763
4,2017-10-28,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0.084503,0.849526,0.047963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104354,2020-03-01,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0.138408,0.513213,0.030329
104355,2009-12-01,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1.039210,0.809993,0.124818
104356,2019-02-18,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0.559830,0.569863,0.005769
104357,2004-02-02,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0.252021,0.904808,0.062211


In [8]:
x_COVID_baseline_not_scaled

,date,total_cases,daily_cases,daily_death,OUT_POSITIVE_QUARANTINE_PSMPD,MO,TU,WE,TH,FR,SA,SU,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ARSON,ASSAULT W/DANGEROUS WEAPON,BURGLARY,HOMICIDE,MOTOR VEHICLE THEFT,ROBBERY,SEX ABUSE,THEFT F/AUTO,THEFT/OTHER,DAY,EVENING,MIDNIGHT,distance_metro,distance_nightclubs,distance_worship
0,2020-09-01,14077.0,28.0,1.0,80.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0.189878,0.352657,0.081604
1,2020-09-01,14077.0,28.0,1.0,80.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0.585539,1.073603,0.156339
2,2020-09-01,14077.0,28.0,1.0,80.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0.490429,0.876416,0.009941
3,2020-08-31,14049.0,57.0,0.0,87.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0.229824,0.360878,0.051530
4,2020-09-01,14077.0,28.0,1.0,80.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0.063730,0.035825,0.076037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43159,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0.418831,0.078911,0.159204
43160,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.718883,0.790578,0.034208
43161,2021-02-03,37365.0,166.0,6.0,106.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.345018,0.900916,0.097899
43162,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0.442868,0.972997,0.004663


In [9]:
x_PRE_COVID_experiment_not_scaled

,date,MO,TU,WE,TH,FR,SA,SU,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ARSON,ASSAULT W/DANGEROUS WEAPON,BURGLARY,HOMICIDE,MOTOR VEHICLE THEFT,ROBBERY,SEX ABUSE,THEFT F/AUTO,THEFT/OTHER,DAY,EVENING,MIDNIGHT,distance_metro,distance_nightclubs,distance_worship
0,2020-09-01,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0.189878,0.352657,0.081604
1,2020-09-01,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0.585539,1.073603,0.156339
2,2020-09-01,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0.490429,0.876416,0.009941
3,2020-08-31,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0.229824,0.360878,0.051530
4,2020-09-01,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0.063730,0.035825,0.076037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43159,2021-02-04,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0.418831,0.078911,0.159204
43160,2021-02-04,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.718883,0.790578,0.034208
43161,2021-02-03,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.345018,0.900916,0.097899
43162,2021-02-04,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0.442868,0.972997,0.004663


In [10]:
x_COVID_experiment_not_scaled

,date,total_cases,daily_cases,daily_death,OUT_POSITIVE_QUARANTINE_PSMPD,MO,TU,WE,TH,FR,SA,SU,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ARSON,ASSAULT W/DANGEROUS WEAPON,BURGLARY,HOMICIDE,MOTOR VEHICLE THEFT,ROBBERY,SEX ABUSE,THEFT F/AUTO,THEFT/OTHER,DAY,EVENING,MIDNIGHT,distance_metro,distance_nightclubs,distance_worship
0,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1.127080,0.674769,0.077614
1,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.191281,0.079607,0.049559
2,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0.083639,0.124886,0.068719
3,2017-10-27,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.135380,0.296590,0.074763
4,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0.084503,0.849526,0.047963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104354,2020-03-01,0.0,0.0,0.0,0.0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0.138408,0.513213,0.030329
104355,2009-12-01,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1.039210,0.809993,0.124818
104356,2019-02-18,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0.559830,0.569863,0.005769
104357,2004-02-02,0.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0.252021,0.904808,0.062211


In [11]:
del x_PRE_COVID_baseline_not_scaled['date']
del x_COVID_baseline_not_scaled['date']
del x_PRE_COVID_experiment_not_scaled['date']
del x_COVID_experiment_not_scaled['date']

In [12]:
'''
# X Data:
x_PRE_COVID_baseline_not_scaled 
x_COVID_baseline_not_scaled 

x_PRE_COVID_experiment_not_scaled 
x_COVID_experiment_not_scaled

# Label Data
y_PRE_COVID_base
y_COVID_base

y_PRE_COVID_experiment
y_COVID_experiment
'''
print("")

In [16]:
def cross_validation(x_PRE_COVID_baseline_not_scaled, y_PRE_COVID_base, x_COVID_baseline_not_scaled, y_COVID_base, 
                     x_PRE_COVID_experiment_not_scaled, y_PRE_COVID_experiment, x_COVID_experiment_not_scaled, y_COVID_experiment,
                     model):   
  # get original data
  random_state = [1, 10, 25, 40, 50]

  ###################
  cross_val_score_lst_PRE_COVID_MODEL_base = []
  pr_lst_PRE_COVID_MODEL_base = [] # list of [Precision, Recall] tuples from each Fold

  cross_val_score_lst_COVID_MODEL_base = []
  pr_lst_COVID_MODEL_base = [] # list of [Precision, Recall] tuples from each Fold

  ###################
  cross_val_score_lst_TEST_ONLY_PRE_COVID_MODEL_ON_COVID_experiment = []
  pr_lst_PRE_COVID_MODEL_ON_COVID_experiment = []

  cross_val_score_lst_TEST_ONLY_COVID_MODEL_ON_PRE_COVID_experiment = []
  pr_lst_COVID_MODEL_ON_PRE_COVID_experiment = []

  ###################
  # go through validation splits
  for val in range(0, 5):
      
      state_idx = val
      ############################
      #### GET Nth VALIDATION ####
      ############################
      # get Nth validation
      # Baseline:
      curr_X_train_PRE_COVID_unscaled_base, curr_X_test_PRE_COVID_unscaled_base, curr_y_train_PRE_COVID_base, curr_y_test_PRE_COVID_base = train_test_split(x_PRE_COVID_baseline_not_scaled, 
                                                                                                                                                            y_PRE_COVID_base,
                                                                                                                                                            test_size=0.3,
                                                                                                                                                            shuffle=True,
                                                                                                                                                            random_state=random_state[state_idx])
      
      curr_X_train_COVID_unscaled_base, curr_X_test_COVID_unscaled_base, curr_y_train_COVID_base, curr_y_test_COVID_base = train_test_split(x_COVID_baseline_not_scaled,
                                                                                                                                            y_COVID_base, 
                                                                                                                                            test_size=0.3, 
                                                                                                                                            shuffle=True,
                                                                                                                                            random_state=random_state[state_idx])
      # Experiment
      curr_X_train_PRE_COVID_unscaled_experiment, curr_X_test_PRE_COVID_unscaled_experiment, curr_y_train_PRE_COVID_experiment, curr_y_test_PRE_COVID_experiment = train_test_split(x_PRE_COVID_experiment_not_scaled,
                                                                                                                                                                                    y_PRE_COVID_experiment,
                                                                                                                                                                                    test_size=0.3, 
                                                                                                                                                                                    shuffle=True,
                                                                                                                                                                                    random_state=random_state[state_idx])
      
      curr_X_train_COVID_unscaled_experiment, curr_X_test_COVID_unscaled_experiment, curr_y_train_COVID_experiment, curr_y_test_COVID_experiment = train_test_split(x_COVID_experiment_not_scaled, 
                                                                                                                                                                    y_COVID_experiment, 
                                                                                                                                                                    test_size=0.3, 
                                                                                                                                                                    shuffle=True,
                                                                                                                                                                    random_state=random_state[state_idx])
      print("Dataset shapes:")
      print("  X_train_PRE_COVID_Baseline:", curr_X_train_PRE_COVID_unscaled_base.shape)
      print("  X_train_PRE_COVID_Experiment:", curr_X_train_PRE_COVID_unscaled_experiment.shape)
      print("")
      print("  X_train_COVID_Baseline:", curr_X_train_COVID_unscaled_base.shape)
      print("  X_train_COVID_Experiment:", curr_X_train_COVID_unscaled_experiment.shape)
      
      # Setup scaler and train on according baseline trains
      MinMax_scaler_PRE_COVID = MinMaxScaler(feature_range=(0, 1))
      MinMax_scaler_PRE_COVID = MinMax_scaler_PRE_COVID.fit(curr_X_train_PRE_COVID_unscaled_base) # Baseline Pre-COVID Train

      MinMax_scaler_COVID = MinMaxScaler(feature_range=(0, 1))
      MinMax_scaler_COVID = MinMax_scaler_COVID.fit(curr_X_train_COVID_unscaled_base) # Baseline COVID Train
      
      ####################
      #### SCALE DATA ####
      ####################
      # Scale the data (Base one, and experiment one)
      # For PRE_COVID:
      curr_x_train_PRE_COVID_base = MinMax_scaler_PRE_COVID.transform(curr_X_train_PRE_COVID_unscaled_base) # For training Pre-COVID model
      curr_x_test_PRE_COVID_base = MinMax_scaler_PRE_COVID.transform(curr_X_test_PRE_COVID_unscaled_base) # For testing Pre-COVID model (base)
      curr_x_test_PRE_COVID_on_COVID_experiment = MinMax_scaler_PRE_COVID.transform(curr_X_test_PRE_COVID_unscaled_experiment) # For testing Pre-COVID on COVID (experiment)

      # For COVID:
      curr_x_train_COVID_base = MinMax_scaler_COVID.transform(curr_X_train_COVID_unscaled_base) # For training COVID model
      curr_x_test_COVID_base = MinMax_scaler_COVID.transform(curr_X_test_COVID_unscaled_base) # For testing COVID model (base)
      curr_x_test_COVID_on_PRE_COVID_experiment = MinMax_scaler_COVID.transform(curr_X_test_COVID_unscaled_experiment) # For testing COVID on Pre-COVID (experiment)
      
      #############################
      #### PRE-COVID PRE-MODEL ####
      #############################
      curr_model_PRE_COVID = model
      #LogisticRegression(multi_class='multinomial', max_iter=500) #SVC(C=41, kernel='rbf') # SET YOUR MODEL HERE WITH ALL SELECTED HYPERPARAMETERS AND OTHER SETTINGS
      curr_model_PRE_COVID.fit(curr_x_train_PRE_COVID_base, curr_y_train_PRE_COVID_base)
      # Get Pre-COVID Baseline:
      curr_accuracy_PRE_COVID_base = accuracy_score(curr_y_test_PRE_COVID_base, 
                                                    curr_model_PRE_COVID.predict(curr_x_test_PRE_COVID_base))
      curr_precision_PRE_COVID_base = precision_score(curr_y_test_PRE_COVID_base, 
                                                      curr_model_PRE_COVID.predict(curr_x_test_PRE_COVID_base),
                                                      average='macro') 
      curr_recall_PRE_COVID_base = recall_score(curr_y_test_PRE_COVID_base, 
                                                curr_model_PRE_COVID.predict(curr_x_test_PRE_COVID_base),
                                                average='macro')
      
      cross_val_score_lst_PRE_COVID_MODEL_base.append(curr_accuracy_PRE_COVID_base)
      pr_lst_PRE_COVID_MODEL_base.append([curr_precision_PRE_COVID_base, curr_recall_PRE_COVID_base])
      
      
      # Get Pre-COVID Experiment:
      curr_accuracy_PRE_COVID_experiment = accuracy_score(curr_y_test_PRE_COVID_experiment, 
                                                          curr_model_PRE_COVID.predict(curr_x_test_PRE_COVID_on_COVID_experiment))
      curr_precision_PRE_COVID_experiment = precision_score(curr_y_test_PRE_COVID_experiment, 
                                                            curr_model_PRE_COVID.predict(curr_x_test_PRE_COVID_on_COVID_experiment),
                                                            average='macro') 
      curr_recall_PRE_COVID_experiment = recall_score(curr_y_test_PRE_COVID_experiment, 
                                                      curr_model_PRE_COVID.predict(curr_x_test_PRE_COVID_on_COVID_experiment),
                                                      average='macro')
      
      cross_val_score_lst_TEST_ONLY_PRE_COVID_MODEL_ON_COVID_experiment.append(curr_accuracy_PRE_COVID_experiment)
      pr_lst_PRE_COVID_MODEL_ON_COVID_experiment.append([curr_precision_PRE_COVID_experiment, curr_recall_PRE_COVID_experiment])
      
      
      #########################
      #### COVID PRE-MODEL ####
      #########################
      curr_model_COVID = model
      #LogisticRegression(multi_class='multinomial', max_iter=500) # SVC(C=41, kernel='rbf') # SET YOUR MODEL HERE WITH ALL SELECTED HYPERPARAMETERS AND OTHER SETTINGS
      curr_model_COVID.fit(curr_x_train_COVID_base, curr_y_train_COVID_base)
      # Get COVID Baseline:
      curr_accuracy_COVID_base = accuracy_score(curr_y_test_COVID_base, 
                                                curr_model_COVID.predict(curr_x_test_COVID_base))
      curr_precision_COVID_base = precision_score(curr_y_test_COVID_base, 
                                                  curr_model_COVID.predict(curr_x_test_COVID_base),
                                                  average='macro') 
      curr_recall_COVID_base = recall_score(curr_y_test_COVID_base, 
                                            curr_model_COVID.predict(curr_x_test_COVID_base),
                                            average='macro')
      
      cross_val_score_lst_COVID_MODEL_base.append(curr_accuracy_COVID_base)
      pr_lst_COVID_MODEL_base.append([curr_precision_COVID_base, curr_recall_COVID_base])
      
      
      # Get COVID Experiment:
      curr_accuracy_COVID_experiment = accuracy_score(curr_y_test_COVID_experiment, 
                                                      curr_model_COVID.predict(curr_x_test_COVID_on_PRE_COVID_experiment))
      curr_precision_COVID_experiment = precision_score(curr_y_test_COVID_experiment, 
                                                        curr_model_COVID.predict(curr_x_test_COVID_on_PRE_COVID_experiment),
                                                        average='macro') 
      curr_recall_COVID_experiment = recall_score(curr_y_test_COVID_experiment, 
                                                  curr_model_COVID.predict(curr_x_test_COVID_on_PRE_COVID_experiment),
                                                  average='macro')
      
      cross_val_score_lst_TEST_ONLY_COVID_MODEL_ON_PRE_COVID_experiment.append(curr_accuracy_COVID_experiment)
      pr_lst_COVID_MODEL_ON_PRE_COVID_experiment.append([curr_precision_COVID_experiment, curr_recall_COVID_experiment])
  return print("\n",
               "PRE-COVID Model Baseline:",
               "Accuracy scores from each fold:", cross_val_score_lst_PRE_COVID_MODEL_base,
               "Precision and Recall from each fold:", pr_lst_PRE_COVID_MODEL_base,
               '\n',
               "COVID Model Baseline:",
               "Accuracy scores from each fold:", cross_val_score_lst_COVID_MODEL_base,
               "Precision and Recall from each fold:", pr_lst_COVID_MODEL_base,
               '\n',
               "PRE-COVID Model Experiment (on COVID Data):",
               "Accuracy scores from each fold:", cross_val_score_lst_TEST_ONLY_PRE_COVID_MODEL_ON_COVID_experiment,
               "Precision and Recall from each fold:", pr_lst_PRE_COVID_MODEL_ON_COVID_experiment,
               '\n',
               "COVID Model Experiment (on Pre-COVID Data):",
               "Accuracy scores from each fold:", cross_val_score_lst_TEST_ONLY_COVID_MODEL_ON_PRE_COVID_experiment,
               "Precision and Recall from each fold:", pr_lst_COVID_MODEL_ON_PRE_COVID_experiment,
               sep='\n')


In [17]:
lr = LogisticRegression(multi_class='multinomial', max_iter=500)

In [18]:
cross_validation(x_PRE_COVID_baseline_not_scaled, y_PRE_COVID_base, 
                 x_COVID_baseline_not_scaled, y_COVID_base, 
                 x_PRE_COVID_experiment_not_scaled, y_PRE_COVID_experiment, 
                 x_COVID_experiment_not_scaled, y_COVID_experiment, lr)

Dataset shapes:
  X_train_PRE_COVID_Baseline: (73051, 34)
  X_train_PRE_COVID_Experiment: (30214, 34)

  X_train_COVID_Baseline: (30214, 38)
  X_train_COVID_Experiment: (73051, 38)
Dataset shapes:
  X_train_PRE_COVID_Baseline: (73051, 34)
  X_train_PRE_COVID_Experiment: (30214, 34)

  X_train_COVID_Baseline: (30214, 38)
  X_train_COVID_Experiment: (73051, 38)
Dataset shapes:
  X_train_PRE_COVID_Baseline: (73051, 34)
  X_train_PRE_COVID_Experiment: (30214, 34)

  X_train_COVID_Baseline: (30214, 38)
  X_train_COVID_Experiment: (73051, 38)
Dataset shapes:
  X_train_PRE_COVID_Baseline: (73051, 34)
  X_train_PRE_COVID_Experiment: (30214, 34)

  X_train_COVID_Baseline: (30214, 38)
  X_train_COVID_Experiment: (73051, 38)
Dataset shapes:
  X_train_PRE_COVID_Baseline: (73051, 34)
  X_train_PRE_COVID_Experiment: (30214, 34)

  X_train_COVID_Baseline: (30214, 38)
  X_train_COVID_Experiment: (73051, 38)


PRE-COVID Model Baseline:
Accuracy scores from each fold:
[0.4381947106170947, 0.437236489076

In [19]:
svc = SVC(C=41, kernel='rbf')

In [20]:
cross_validation(x_PRE_COVID_baseline_not_scaled, y_PRE_COVID_base, 
                 x_COVID_baseline_not_scaled, y_COVID_base, 
                 x_PRE_COVID_experiment_not_scaled, y_PRE_COVID_experiment, 
                 x_COVID_experiment_not_scaled, y_COVID_experiment, svc)

Dataset shapes:
  X_train_PRE_COVID_Baseline: (73051, 34)
  X_train_PRE_COVID_Experiment: (30214, 34)

  X_train_COVID_Baseline: (30214, 38)
  X_train_COVID_Experiment: (73051, 38)
Dataset shapes:
  X_train_PRE_COVID_Baseline: (73051, 34)
  X_train_PRE_COVID_Experiment: (30214, 34)

  X_train_COVID_Baseline: (30214, 38)
  X_train_COVID_Experiment: (73051, 38)


KeyboardInterrupt: ignored